Bayesian Statistics Made Simple
===

Code and exercises from my workshop on Bayesian statistics in Python.

Copyright 2016 Allen Downey

MIT License: https://opensource.org/licenses/MIT

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd

import seaborn as sns
sns.set_style('white')
sns.set_context('talk')

import matplotlib.pyplot as plt

In [ ]:
def underride(d, **options):
    """Add key-value pairs to d only if key is not in d.

    d: dictionary
    options: keyword args to add to d
    
    returns: modified d
    """
    for key, val in options.items():
        d.setdefault(key, val)

    return d

In [ ]:
class Pmf(pd.Series):
    """Represents a probability Mass Function (PMF)."""

    def __init__(self, *args, **kwargs):
        """Initialize a Series.

        Note: this cleans up a weird Series behavior, which is
        that Series() and Series([]) yield different results.
        See: https://github.com/pandas-dev/pandas/issues/16737
        """
        if args:
            super().__init__(*args, **kwargs)
        else:
            underride(kwargs, dtype=np.float64)
            super().__init__([], **kwargs)
            
    @property
    def qs(self):
        """Get the quantities.
        
        returns: NumPy array
        """
        return self.index.values

    @property
    def ps(self):
        """Get the probabilities.
        
        returns: NumPy array
        """
        return self.values
            
    def _repr_html_(self):
        """Returns an HTML representation of the series.

        Mostly used for Jupyter notebooks.
        """
        df = pd.DataFrame(dict(probs=self))
        return df._repr_html_()

### Working with Pmfs

Create a Pmf object to represent a six-sided die.

In [ ]:
d6 = Pmf()

A Pmf is a map from possible outcomes to their probabilities.

In [ ]:
for x in [1,2,3,4,5,6]:
    d6[x] = 1

Initially the probabilities don't add up to 1.

In [ ]:
d6

`normalize` adds up the probabilities and divides through.  The return value is the total probability before normalizing.

In [ ]:
def normalize(self):
    """Make the probabilities add up to 1 (modifies self).
    
    returns: normlizing constant
    """
    total = self.sum()
    self /= total
    return total

Pmf.normalize = normalize

In [ ]:
d6.normalize()

Now the Pmf is normalized.

In [ ]:
d6

And we can compute its mean (which only works if it's normalized).

In [ ]:
def mean(self):
    """Computes expected value.
    
    returns: float
    """
    return np.sum(self.ps * self.qs)

Pmf.mean = mean

In [ ]:
d6.mean()

`choice` chooses a random values from the Pmf.

In [ ]:
def choice(self, **options):
    """Makes a random choice.
    
    options: same as np.random.choice
    
    returns: NumPy array
    """
    options['p'] = self.ps
    return np.random.choice(self.qs, **options)

Pmf.choice = choice

In [ ]:
d6.choice(size=10)

`bar` plots the Pmf as a bar chart

In [ ]:
def bar(self, **options):
    """Makes a bar plot.
    
    options: same as plt.bar
    """
    underride(options, label=self.name)
    plt.bar(self.qs, self.ps, **options)

Pmf.bar = bar

In [ ]:
def decorate_dice(title):
    """Labels the axes.
    
    title: string
    """
    plt.xlabel('Outcome')
    plt.ylabel('PMF')
    plt.title(title)

In [ ]:
d6.bar()
decorate_dice('One die')

**Exercise 1:**  Suppose we roll two 6-sided dice and add them up.  The following function computes the `Pmf` of the sum.

It uses an outer sum and an outer product in a non-obvious way.  You don't have to understand how it works, but you might find it interesting.

In [ ]:
def pmf_add(pmf1, pmf2):
    """Distribution of the sum.

    pmf1: Pmf
    pmf2: Pmf

    returns: new Pmf
    """
    qs = np.add.outer(pmf1.qs, pmf2.qs).flatten()
    ps = np.multiply.outer(pmf1.ps, pmf2.ps).flatten()
    series = pd.Series(ps).groupby(qs).sum()
    return Pmf(series)

By defining `__add__`, we can use the `+` operator to "add" two `Pmf` objects.

In [ ]:
def __add__(self, x):
    """Computes the Pmf of the sum of values drawn from self and x.

    x: another Pmf or a scalar

    returns: new Pmf
    """
    if isinstance(x, Pmf):
        return pmf_add(self, x)
    else:
        return Pmf(self.qs + x, self.ps)

Pmf.__add__ = __add__
Pmf.__radd__ = __add__

Compute the distribution of the sum, plot it, and compute its mean.

In [ ]:
# Solution goes here

In [ ]:
# Solution goes here

**Exercise 2:** Suppose I roll two dice and tell you the result is greater than 3.

Plot the Pmf of the remaining possible outcomes and compute its mean.

In [ ]:
# Solution goes here

### The cookie problem

The following function make a `Pmf` object from a sequence of values.

In [ ]:
def pmf_from_seq(seq, normalize=True, sort=True, **options):
    """Make a PMF from a sequence of values.
    
    seq: any kind of sequence
    normalize: whether to normalize the Pmf, default True
    sort: whether to sort the Pmf by values, default True
    options: passed to the pd.Series constructor
    
    returns: Pmf object
    """
    series = pd.Series(seq).value_counts(sort=False)
    
    options['copy'] = False
    pmf = Pmf(series, **options)
    
    if sort:
        pmf.sort_index(inplace=True)

    if normalize:
        pmf.normalize()
    
    return pmf

We'll use `pmf_from_seq` to create a Pmf with two equally likely hypotheses.

In [ ]:
cookie = pmf_from_seq(['Bowl 1', 'Bowl 2'])
cookie

The update each hypothesis with the likelihood of the data (a vanilla cookie).

In [ ]:
cookie['Bowl 1'] *= 0.75
cookie['Bowl 2'] *= 0.5
cookie.normalize()

And display the posterior probabilities.

In [ ]:
cookie

**Exercise 3:** Suppose we put the first cookie back, stir, choose again from the same bowl, and get a chocolate cookie.  What aer the posterior probabilities after the second cookie?

Hint: The posterior (after the first cookie) becomes the prior (before the second cookie).

In [ ]:
# Solution goes here

**Exercise 4:** Instead of doing two updates, what if we collapse the two pieces of data into one update?

Re-initialize `Pmf` with two equally likely hypotheses and perform one update based on two pieces of data, a vanilla cookie and a chocolate cookie.

The result should be the same regardless of how many updates you do (or the order of updates).

In [ ]:
# Solution goes here

### The dice problem

Create a Suite to represent dice with different numbers of sides.

In [ ]:
pmf = pmf_from_seq([4, 6, 8, 12])
pmf

**Exercise 5:** We'll solve this problem two ways.  First we'll do it "by hand", as we did with the cookie problem; that is, we'll multiply each hypothesis by the likelihood of the data, and then renormalize.

In the space below, update `pmf` based on the likelihood of the data (rolling a 6), then normalize and print the results.

In [ ]:
# Solution goes here

**Exercise 6:**  Now let's do the same calculation using `update`, which encodes the structure of a Bayesian update:

In [ ]:
def update(self, likelihood, data):
    """Bayesian update.
    
    likelihood: function that takes (data, hypo) and returns
                likelihood of data under hypo
    data: whatever format like_func understands
    
    returns: normalizing constant
    """
    for hypo in self.qs:
        self[hypo] *= likelihood(data, hypo)
        
    return self.normalize()

Pmf.update = update

Define a function called `likelihood_dice` that takes `data` and `hypo` and returns the probability of the data (the outcome of rolling the die) for a given hypothesis (number of sides on the die).

Hint: What should you do if the outcome exceeds the hypothetical number of sides on the die?

Here's an outline to get you started.

In [ ]:
def likelihood_dice(data, hypo):
    """Likelihood function for the dice problem.
    
    data: outcome of the die roll
    hypo: number of sides
    
    returns: float probability
    """
    # TODO: fill this in!
    return 1

In [ ]:
# Solution goes here

Now we can create a `Pmf` object and update it.

In [ ]:
dice = pmf_from_seq([4, 6, 8, 12])
dice.update(likelihood_dice, 6)
dice

If we get more data, we can perform more updates.

In [ ]:
for roll in [8, 7, 7, 5, 4]:
    dice.update(likelihood_dice, roll)

Here are the results.

In [ ]:
dice

### The German tank problem

The German tank problem is actually identical to the dice problem.

In [ ]:
def likelihood_tank(data, hypo):
    """Likelihood function for the tank problem.
    
    data: observed serial number
    hypo: number of tanks
    
    returns: float probability
    """
    if data > hypo:
        return 0
    else:
        return 1 / hypo

Here is the update after seeing Tank #42.

In [ ]:
tank = pmf_from_seq(range(100))
tank.update(likelihood_tank, 42)
tank.mean()

And here's what the posterior distribution looks like.

In [ ]:
def decorate_tank(title):
    """Labels the axes.
    
    title: string
    """
    plt.xlabel('Number of tanks')
    plt.ylabel('PMF')
    plt.title(title)

In [ ]:
tank.plot()
decorate_tank('Distribution after one tank')

**Exercise 7:**  Suppose we see another tank with serial number 17.  What effect does this have on the posterior probabilities?

Update the `Pmf` with the new data and plot the results.

In [ ]:
# Solution goes here

### The Euro problem

**Exercise 8:**  Write a function called `likelihood_euro` that defines the likelihood function for the Euro problem.  Note that `hypo` is in the range 0 to 100.

Here's an outline to get you started.

In [ ]:
def likelihood_euro(data, hypo):
    """ Likelihood function for the Euro problem.
    
    data: string, either 'H' or 'T'
    hypo: prob of heads (0-100)
    
    returns: float probability
    """
    # TODO: fill this in!
    return 1

In [ ]:
# Solution goes here

For the prior, we'll start with a uniform distribution from 0 to 100.

In [ ]:
def decorate_euro(title):
    """Labels the axes.
    
    title: string
    """
    plt.xlabel('Probability of heads')
    plt.ylabel('PMF')
    plt.title(title)

In [ ]:
euro = pmf_from_seq(range(101))
euro.plot()
decorate_euro('Prior distribution')

Now we can update with a single heads:

In [ ]:
euro.update(likelihood_euro, 'H')
euro.plot()
decorate_euro('Posterior distribution, one heads')

Another heads:

In [ ]:
euro.update(likelihood_euro, 'H')
euro.plot()
decorate_euro('Posterior distribution, two heads')

And a tails:

In [ ]:
euro.update(likelihood_euro, 'T')
euro.plot()
decorate_euro('Posterior distribution, HHT')

Starting over, here's what it looks like after 7 heads and 3 tails.

In [ ]:
euro = pmf_from_seq(range(101))

for outcome in 'HHHHHHHTTT':
    euro.update(likelihood_euro, outcome)

euro.plot()
decorate_euro('Posterior distribution, 7 heads, 3 tails')

In [ ]:
def MAP(self):
    """Maximum aposteori probability.
    
    returns: the value with the highest probability
    """
    return self.idxmax()

Pmf.MAP = MAP

In [ ]:
euro.MAP()

The maximum posterior probability is 70%, which is the observed proportion.

Here are the posterior probabilities after 140 heads and 110 tails.

In [ ]:
euro = pmf_from_seq(range(101))

evidence = 'H' * 140 + 'T' * 110
for outcome in evidence:
    euro.update(likelihood_euro, outcome)
    
euro.plot()

decorate_euro('Posterior distribution, 140 heads, 110 tails')

The posterior mean s about 56%

In [ ]:
euro.mean()

So is the value with Maximum Aposteriori Probability (MAP).

In [ ]:
euro.MAP()

The posterior credible interval has a 90% chance of containing the true value (provided that the prior distribution truly represents our background knowledge).

In [ ]:
from scipy.interpolate import interp1d

def quantile(self, ps):
    """Quantities corresponding to given probabilities.
    
    ps: sequence of probabilities
    
    return: sequence of quantities
    """
    cdf = self.sort_index().cumsum()
    interp = interp1d(cdf.values, cdf.index,
                      kind='next',
                      copy=False,
                      assume_sorted=True,
                      bounds_error=False,
                      fill_value=(self.qs[0], np.nan))
    return interp(ps)

Pmf.quantile = quantile

In [ ]:
euro.quantile(0.5)

In [ ]:
def credible_interval(self, p):
    """Credible interval containing the given probability.
    
    p: float 0-1
    
    returns: array of two quantities
    """
    tail = (1-p) / 2
    ps = [tail, 1-tail]
    return self.quantile(ps)

Pmf.credible_interval = credible_interval

In [ ]:
euro.credible_interval(0.9)

### Swamping the prior

The following function makes a Euro object with a triangle prior.

In [ ]:
def TrianglePrior():
    """Makes a Suite with a triangular prior.
    """
    suite = Pmf(name='triangle')
    for x in range(0, 51):
        suite[x] = x
    for x in range(51, 101):
        suite[x] = 100-x 
    suite.normalize()
    return suite

And here's what it looks like:

In [ ]:
euro1 = pmf_from_seq(range(101), name='uniform')
euro1.plot()

euro2 = TrianglePrior()
euro2.plot()

plt.legend()
decorate_euro('Prior distributions')

**Exercise 9:** Update `euro1` and `euro2` with the same data we used before (140 heads and 110 tails) and plot the posteriors.  How big is the difference in the means?

In [ ]:
# Solution goes here

The posterior distributions are not identical, but with this data, they converge to the point where there is no practical difference, for most purposes.